<a href="https://colab.research.google.com/github/vivekdarji-git/public/blob/master/tfds_hub_simple_neural_network_movie_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
tf.__version__


'2.3.0'

In [ ]:
#Get all TF datasets
#tfds.list_builders()

In [4]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                   split=('train[:60%]', 'train[40%:]', 'test'), 
                                                   as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY49D5J/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY49D5J/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY49D5J/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [15]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [7]:
#View Sample Data
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [8]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [16]:
#convert sentences using word2vec (a separated readily trained neural network will be used - tensorflow hub)
#will use gnews-swivel-20dim to convert text into embedding vector

hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1",
                           input_shape=[], 
                           dtype=tf.string,
                           trainable = True)

In [17]:
hub_layer(train_examples_batch[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

In [18]:
train_examples_batch[:2]

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [22]:
#creating network - model 1

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1 , activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#model 2 using rnn (to change embedding layer config)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    model.add(hub_layer)
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])



In [27]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [28]:
model.fit(train_data.shuffle(10000).batch(512),
          epochs = 20,
          validation_data = validation_data.batch(512),
          verbose = True)

Epoch 1/20
30/30 [==============================] - 3s 94ms/step - loss: 0.7702 - accuracy: 0.4985 - val_loss: 0.6616 - val_accuracy: 0.6165
Epoch 2/20
30/30 [==============================] - 3s 90ms/step - loss: 0.6287 - accuracy: 0.6597 - val_loss: 0.6000 - val_accuracy: 0.6945
Epoch 3/20
30/30 [==============================] - 3s 89ms/step - loss: 0.5836 - accuracy: 0.7085 - val_loss: 0.5701 - val_accuracy: 0.7173
Epoch 4/20
30/30 [==============================] - 3s 92ms/step - loss: 0.5475 - accuracy: 0.7361 - val_loss: 0.5455 - val_accuracy: 0.7231
Epoch 5/20
30/30 [==============================] - 3s 92ms/step - loss: 0.5054 - accuracy: 0.7663 - val_loss: 0.4946 - val_accuracy: 0.7692
Epoch 6/20
30/30 [==============================] - 3s 89ms/step - loss: 0.4730 - accuracy: 0.7830 - val_loss: 0.4652 - val_accuracy: 0.7895
Epoch 7/20
30/30 [==============================] - 3s 94ms/step - loss: 0.4429 - accuracy: 0.8005 - val_loss: 0.4508 - val_accuracy: 0.7968
Epoch 8/20
30

In [29]:
model.predict(["this is the worst movie i have ever seen"])

array([[0.1292317]], dtype=float32)